In [1]:
import os
import sys
import numpy as np
import time
import torch
import utils_imagenet
import glob
import random
import logging
import argparse
import torch.nn as nn
import genotypes
import torch.utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn

from torch.autograd import Variable
from model import NetworkImageNet as Network


parser = argparse.ArgumentParser("imagenet")
parser.add_argument('--data', type=str, default='../data/imagenet/', help='location of the data corpus')
parser.add_argument('--batch_size', type=int, default=128, help='batch size')
parser.add_argument('--learning_rate', type=float, default=0.1, help='init learning rate')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
parser.add_argument('--weight_decay', type=float, default=3e-5, help='weight decay')
parser.add_argument('--report_freq', type=float, default=100, help='report frequency')
parser.add_argument('--gpu', type=int, default=0, help='gpu device id')
parser.add_argument('--epochs', type=int, default=250, help='num of training epochs')
parser.add_argument('--init_channels', type=int, default=48, help='num of init channels')
parser.add_argument('--layers', type=int, default=14, help='total number of layers')
parser.add_argument('--auxiliary', action='store_true', default=True, help='use auxiliary tower')
parser.add_argument('--auxiliary_weight', type=float, default=0.4, help='weight for auxiliary loss')
parser.add_argument('--drop_path_prob', type=float, default=0, help='drop path probability')
parser.add_argument('--save', type=str, default='EXP', help='experiment name')
parser.add_argument('--seed', type=int, default=0, help='random seed')
parser.add_argument('--arch', type=str, default='DARTS', help='which architecture to use')
parser.add_argument('--grad_clip', type=float, default=5., help='gradient clipping')
parser.add_argument('--label_smooth', type=float, default=0.1, help='label smoothing')
parser.add_argument('--gamma', type=float, default=0.97, help='learning rate decay')
parser.add_argument('--decay_period', type=int, default=1, help='epochs between two learning rate decays')
parser.add_argument('--parallel', action='store_true', default=False, help='data parallelism')
args = parser.parse_args([])


CLASSES = 1000

In [2]:
np.random.seed(args.seed)
torch.cuda.set_device(args.gpu)
cudnn.benchmark = True
torch.manual_seed(args.seed)
cudnn.enabled=True
torch.cuda.manual_seed(args.seed)
logging.info('gpu device = %d' % args.gpu)
logging.info("args = %s", args)


In [6]:
genotype = eval("genotypes.%s" % args.arch)
pre_model = Network(args.init_channels, CLASSES, args.layers, args.auxiliary, genotype)
pre_model = pre_model.cuda()
#pre_model.load_state_dict(torch.load('model_best.pth.tar')['state_dict'])

pre_model.load_state_dict(torch.load('DARTS_imagenet_model.pt',map_location='cuda:0')['state_dict'])


48 48 48
48 192 48
192 192 48
192 192 48
192 192 96
192 384 96
384 384 96
384 384 96
384 384 96
384 384 192
384 768 192
768 768 192
768 768 192
768 768 192


<All keys matched successfully>

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
from PIL import ImageFile
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import numpy as np
import pandas as pd
import os
import random 
from shutil import copyfile
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import re
import albumentations as albu
from albumentations.pytorch import ToTensor
from catalyst.data import Augmentor
import torchxrayvision as xrv

/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboar

In [8]:
import torch
import torchvision
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from sklearn.metrics import roc_auc_score
import re
import albumentations as albu
from albumentations.pytorch import ToTensor
from catalyst.data import Augmentor
from skimage.io import imread, imsave
import skimage

In [9]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop((224),scale=(0.5,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

val_transformer = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])


# In[80]:


batchsize=10
def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data

class CovidCTDataset(Dataset):
    def __init__(self, root_dir, txt_COVID, txt_NonCOVID, transform=None):
        """
        Args:
            txt_path (string): Path to the txt file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        File structure:
        - root_dir
            - CT_COVID
                - img1.png
                - img2.png
                - ......
            - CT_NonCOVID
                - img1.png
                - img2.png
                - ......
        """
        self.root_dir = root_dir
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.num_cls = len(self.classes)
        self.img_list = []
        for c in range(self.num_cls):
            cls_list = [[os.path.join(self.root_dir,self.classes[c],item), c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.img_list[idx][0]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        sample = {'img': image,
                  'label': int(self.img_list[idx][1])}
        return sample


In [10]:
trainset = CovidCTDataset(root_dir='/tmp/mozilla_taoliu0/COVID-CT-master/Images-processed',
                          txt_COVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/COVID/trainCT_COVID.txt',
                          txt_NonCOVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/NonCOVID/trainCT_NonCOVID.txt',
                          transform= train_transformer)
valset = CovidCTDataset(root_dir='/tmp/mozilla_taoliu0/COVID-CT-master/Images-processed',
                          txt_COVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/COVID/valCT_COVID.txt',
                          txt_NonCOVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/NonCOVID/valCT_NonCOVID.txt',
                          transform= val_transformer)
testset = CovidCTDataset(root_dir='/tmp/mozilla_taoliu0/COVID-CT-master/Images-processed',
                          txt_COVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/COVID/testCT_COVID.txt',
                          txt_NonCOVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/NonCOVID/testCT_NonCOVID.txt',
                          transform= val_transformer)
print(trainset.__len__())
print(valset.__len__())
print(testset.__len__())

train_loader = DataLoader(trainset, batch_size=batchsize, drop_last=False, shuffle=True,pin_memory=True, num_workers=4)
val_loader = DataLoader(valset, batch_size=batchsize, drop_last=False, shuffle=False,pin_memory=True, num_workers=4)
test_loader = DataLoader(testset, batch_size=batchsize, drop_last=False, shuffle=False,pin_memory=True, num_workers=4)

424
116
200


In [11]:
alpha = None
device = 'cuda'

# In[152]:


def test(epoch):
    
    model.eval()
    test_loss = 0
    correct = 0
    results = []
    
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    
    
    criteria = nn.CrossEntropyLoss()
    # Don't update model
    with torch.no_grad():
        tpr_list = []
        fpr_list = []
        
        predlist=[]
        scorelist=[]
        targetlist=[]
        # Predict
        for batch_index, batch_samples in enumerate(test_loader):
            data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
            data = data[:, 0, :, :]
            data = data[:, None, :, :]
            data = data.repeat(1,3,1,1)
#             print(target)
            output = model(data)
            
            test_loss += criteria(output, target.long())
            score = F.softmax(output, dim=1)
            pred = output.argmax(dim=1, keepdim=True)
#             print('target',target.long()[:, 2].view_as(pred))
            correct += pred.eq(target.long().view_as(pred)).sum().item()
#             TP += ((pred == 1) & (target.long()[:, 2].view_as(pred).data == 1)).cpu().sum()
#             TN += ((pred == 0) & (target.long()[:, 2].view_as(pred) == 0)).cpu().sum()
# #             # FN    predict 0 label 1
#             FN += ((pred == 0) & (target.long()[:, 2].view_as(pred) == 1)).cpu().sum()
# #             # FP    predict 1 label 0
#             FP += ((pred == 1) & (target.long()[:, 2].view_as(pred) == 0)).cpu().sum()
#             print(TP,TN,FN,FP)
            
            
#             print(output[:,1].cpu().numpy())
#             print((output[:,1]+output[:,0]).cpu().numpy())
#             predcpu=(output[:,1].cpu().numpy())/((output[:,1]+output[:,0]).cpu().numpy())
            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
           
    return targetlist, scorelist, predlist

In [13]:
modelname = 'DARTS_Model'

In [14]:
model = Network(args.init_channels, 2, args.layers, args.auxiliary, genotype)
model = torch.nn.DataParallel(model)
new_model = model.cuda()
model.to(device)

48 48 48
48 192 48
192 192 48
192 192 48
192 192 96
192 384 96
384 384 96
384 384 96
384 384 96
384 384 192
384 768 192
768 768 192
768 768 192
768 768 192


DataParallel(
  (module): NetworkImageNet(
    (stem0): Sequential(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (stem1): Sequential(
      (0): ReLU(inplace=True)
      (1): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (cells): ModuleList(
      (0): Cell(
        (preprocess0): FactorizedReduce(
          (relu): ReLU()
          (conv_1): Conv2d(48, 24, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (conv_2): Conv2d(48, 24, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (bn): BatchNor

In [15]:
#pretrained_dict=torch.load('model_best.pth.tar')
pre_model = torch.nn.DataParallel(pre_model)

In [16]:
newmodel_dict=model.state_dict()
premodel_dict=pre_model.state_dict()
new_list=list(newmodel_dict.keys())
pre_list=list(premodel_dict.keys())
for i in range(952):####The model contains 956 keys, we need exclude the last layer
    newmodel_dict[new_list[i]]=premodel_dict[pre_list[i]]

model.load_state_dict(newmodel_dict)

model=model.module
#dict_name=list(model.state_dict())
#for i,p in enumerate(dict_name):
#    print(i,p)

In [18]:

def train(train_loader, model, criterion, optimizer):
    objs = utils_imagenet.AvgrageMeter()
    top1 = utils_imagenet.AvgrageMeter()
    model.train()
    
    loss = 0
    train_correct = 0

    for step, batch_samples in enumerate(train_loader):
        
        input, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
        input = input[:, 0, :, :]
        input = input[:, None, :, :]
#         data, targets_a, targets_b, lam = mixup_data(data, target.long(), alpha, use_cuda=True)
        input = input.repeat(1,3,1,1)        
        
        
        target = target.cuda(async=True)
        input = input.cuda()
        input = Variable(input)
        target = Variable(target)

        optimizer.zero_grad()
        logits, logits_aux = model(input)
        loss = criterion(logits, target)
        if args.auxiliary:
            loss_aux = criterion(logits_aux, target)
            loss += args.auxiliary_weight*loss_aux

        loss.backward()
        nn.utils.clip_grad_norm(model.parameters(), args.grad_clip)
        optimizer.step()

        prec1, prec2 = utils_imagenet.accuracy(logits, target, topk=(1,2))
        n = input.size(0)
        objs.update(loss.data, n)
        top1.update(prec1.data, n)
       # top2.update(prec5.data, n)

        if step % args.report_freq == 0:
            logging.info('train %03d %e %f', step, objs.avg, top1.avg)
            


        pred = logits.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.long().view_as(pred)).sum().item()
    
        # Display progress and write to tensorboard
        if step % bs == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, step, len(train_loader),
                100.0 * step / len(train_loader), loss.item()/ bs))
    
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
        100.0 * train_correct / len(train_loader.dataset)))
    f = open('model_result/{}.txt'.format(modelname), 'a+')
    f.write('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
        100.0 * train_correct / len(train_loader.dataset)))
    f.write('\n')
    f.close()
            
            

    return top1.avg, objs.avg    
    
    
def infer(val_loader, model, criterion,epoch):
    
    objs = utils_imagenet.AvgrageMeter()
    top1 = utils_imagenet.AvgrageMeter()
    top5 = utils_imagenet.AvgrageMeter()
    
    model.eval()
    test_loss = 0
    correct = 0
    results = []
    
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    
    
    # Don't update model
    with torch.no_grad():
        tpr_list = []
        fpr_list = []
        
        predlist=[]
        scorelist=[]
        targetlist=[]
        # Predict
        
    
        for batch_index, batch_samples in enumerate(val_loader):
            data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
            data = data[:, 0, :, :]
            data = data[:, None, :, :]
            data = data.repeat(1,3,1,1)
            data = Variable(data, volatile=True).cuda()
            target = Variable(target, volatile=True).cuda(async=True)            
            
            
            output, logits_aux = model(data)
            
            test_loss += criterion(output, target)
            score = F.softmax(output, dim=1)
            pred = output.argmax(dim=1, keepdim=True)
#             print('target',target.long()[:, 2].view_as(pred))
            correct += pred.eq(target.long().view_as(pred)).sum().item()
            
#             print(output[:,1].cpu().numpy())
#             print((output[:,1]+output[:,0]).cpu().numpy())
#             predcpu=(output[:,1].cpu().numpy())/((output[:,1]+output[:,0]).cpu().numpy())
            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
           
          
    return targetlist, scorelist, predlist





class CrossEntropyLabelSmooth(nn.Module):

    def __init__(self, num_classes, epsilon):
        super(CrossEntropyLabelSmooth, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, inputs, targets):
        log_probs = self.logsoftmax(inputs)
        targets = torch.zeros_like(log_probs).scatter_(1, targets.unsqueeze(1), 1)
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (-targets * log_probs).mean(0).sum()
        return loss
    

bs = 10
votenum = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(valset.__len__())
vote_score = np.zeros(valset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
#scheduler = StepLR(optimizer, step_size=1)

total_epoch = 3000
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
criterion_smooth = CrossEntropyLabelSmooth(CLASSES, args.label_smooth)
criterion_smooth = criterion_smooth.cuda()

In [ ]:
for epoch in range(total_epoch):

    scheduler.step()
    model.drop_path_prob = args.drop_path_prob * epoch / args.epochs
    train_acc, train_obj = train(train_loader, model, criterion_smooth, optimizer)
    targetlist, scorelist, predlist = infer(val_loader, model, criterion_smooth,epoch)
    
    print('target',targetlist)
    print('score',scorelist)
    print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        vote_score = vote_score/votenum
        
        print('vote_pred', vote_pred)
        print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUCp', roc_auc_score(targetlist, vote_pred))
        print('AUC', AUC)
        
        
        
#         if epoch == total_epoch:
        torch.save(model.state_dict(), "model_backup/{}.pt".format(modelname))  
        
        vote_pred = np.zeros(valset.__len__())
        vote_score = np.zeros(valset.__len__())
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        f = open('model_result/{}.txt'.format(modelname), 'a+')
        f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))
        f.close()     
    

Train Epoch: 0 [0/43 (0%)]	Train Loss: 0.086589
Train Epoch: 0 [10/43 (23%)]	Train Loss: 0.082616
Train Epoch: 0 [20/43 (47%)]	Train Loss: 0.079588
Train Epoch: 0 [30/43 (70%)]	Train Loss: 0.065690
Train Epoch: 0 [40/43 (93%)]	Train Loss: 0.048338

Train set: Average loss: 0.0032, Accuracy: 302/424 (71%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [0.78265762 0.7553367  0.76479697 0.62797612 0.57338947 0.66108179
 0.70876461 0.73932266 0.65130717 0.6504243  0.57405186 0.64098626
 0.70083487 0.69505757 0.66451705 0.54386461 0.53634298 0.68464261
 0.50072676 0.76579016 0.7734012  0.72113246 0.58321965 0.78650421
 0.71059954 0.66066408 0.82955045 0.71571118 0.6917522  0.

Train Epoch: 3 [0/43 (0%)]	Train Loss: 0.043385
Train Epoch: 3 [10/43 (23%)]	Train Loss: 0.041744
Train Epoch: 3 [20/43 (47%)]	Train Loss: 0.052528
Train Epoch: 3 [30/43 (70%)]	Train Loss: 0.054613
Train Epoch: 3 [40/43 (93%)]	Train Loss: 0.029317

Train set: Average loss: 0.0004, Accuracy: 388/424 (92%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [8.80712760e-04 7.06256568e-01 2.17018679e-01 5.20902053e-02
 3.79252364e-04 1.77325812e-04 7.35539675e-01 1.21693268e-01
 3.23414890e-04 1.50394430e-02 2.31664851e-02 1.68654624e-05
 1.21466145e-02 4.22601443e-05 1.41527362e-05 2.57640227e-06
 4.01817786e-04 9.74617228e-02 1.84665481e-03 3.79795022e-02
 2.34590136e-02 2.215

Train Epoch: 6 [0/43 (0%)]	Train Loss: 0.014738
Train Epoch: 6 [10/43 (23%)]	Train Loss: 0.012614
Train Epoch: 6 [20/43 (47%)]	Train Loss: 0.006186
Train Epoch: 6 [30/43 (70%)]	Train Loss: 0.019828
Train Epoch: 6 [40/43 (93%)]	Train Loss: 0.007308

Train set: Average loss: 0.0001, Accuracy: 393/424 (93%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [4.17300470e-07 7.71291852e-01 8.37918445e-02 1.86515134e-02
 3.89926527e-07 9.28920690e-06 8.05240154e-01 2.50798706e-02
 8.42418922e-06 3.02599440e-03 1.34313377e-02 1.27021503e-05
 4.93727252e-02 2.20066049e-05 8.66608843e-07 8.21408079e-08
 6.10814663e-04 1.34872526e-01 2.88707175e-04 3.79175469e-02
 5.90248324e-04 1.373

Train Epoch: 9 [0/43 (0%)]	Train Loss: 0.010700
Train Epoch: 9 [10/43 (23%)]	Train Loss: 0.001076
Train Epoch: 9 [20/43 (47%)]	Train Loss: 0.024540
Train Epoch: 9 [30/43 (70%)]	Train Loss: 0.027152
Train Epoch: 9 [40/43 (93%)]	Train Loss: 0.024049

Train set: Average loss: 0.0001, Accuracy: 395/424 (93%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [2.94721804e-06 8.32545280e-01 1.35125443e-01 2.53247190e-02
 3.47989726e-06 1.95817174e-05 9.14246380e-01 6.81327507e-02
 3.40221231e-05 4.45152968e-02 8.93172249e-02 1.06086285e-04
 2.02603608e-01 3.25470173e-05 3.81569043e-06 1.79416446e-07
 7.82749674e-04 4.80327994e-01 4.51838132e-04 1.87075734e-02
 2.65796692e-03 2.924

Train Epoch: 12 [0/43 (0%)]	Train Loss: 0.073252
Train Epoch: 12 [10/43 (23%)]	Train Loss: 0.015270
Train Epoch: 12 [20/43 (47%)]	Train Loss: 0.005173
Train Epoch: 12 [30/43 (70%)]	Train Loss: 0.019806
Train Epoch: 12 [40/43 (93%)]	Train Loss: 0.004938

Train set: Average loss: 0.0003, Accuracy: 401/424 (95%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [9.88456918e-07 6.87983871e-01 2.59953827e-01 1.23025766e-02
 9.45885517e-07 5.19734476e-06 9.04018104e-01 6.42210096e-02
 6.49969661e-06 6.89050881e-03 7.48499390e-03 3.92624534e-05
 1.08494051e-02 1.38701042e-04 1.84212865e-06 1.58220217e-07
 1.19333621e-03 3.67026538e-01 6.98793854e-04 7.10147293e-03
 2.08331575e-03 

...

Train Epoch: 18 [0/43 (0%)]	Train Loss: 0.025688
Train Epoch: 18 [10/43 (23%)]	Train Loss: 0.049345
Train Epoch: 18 [20/43 (47%)]	Train Loss: 0.012424
Train Epoch: 18 [30/43 (70%)]	Train Loss: 0.010303
Train Epoch: 18 [40/43 (93%)]	Train Loss: 0.007077

Train set: Average loss: 0.0007, Accuracy: 385/424 (91%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [1.82113971e-03 2.62453616e-01 1.35332812e-03 3.41813313e-03
 8.35101309e-05 2.02381955e-07 7.66477548e-03 3.05840839e-03
 2.19537265e-04 9.41571197e-04 4.28285748e-02 1.38424783e-08
 2.99620139e-03 2.81950738e-03 1.65180815e-03 6.37073754e-06
 4.97981399e-01 7.91499019e-02 1.11741365e-05 2.86549125e-02
 1.18669472e-03 

Train Epoch: 21 [0/43 (0%)]	Train Loss: 0.010554
Train Epoch: 21 [10/43 (23%)]	Train Loss: 0.000343
Train Epoch: 21 [20/43 (47%)]	Train Loss: 0.000379
Train Epoch: 21 [30/43 (70%)]	Train Loss: 0.001914
Train Epoch: 21 [40/43 (93%)]	Train Loss: 0.013995

Train set: Average loss: 0.0004, Accuracy: 407/424 (96%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [7.79323727e-07 1.76329445e-02 2.23679338e-02 1.11238216e-03
 2.61968580e-08 1.21395267e-08 2.19586268e-02 3.49971349e-04
 5.32380113e-08 9.47244928e-07 7.40417419e-03 8.97256047e-09
 3.55889834e-03 4.10050925e-05 4.02508653e-04 4.42155229e-10
 9.48808236e-07 1.24462716e-01 4.65264253e-04 1.35508537e-01
 7.26460130e-05 

Train Epoch: 24 [0/43 (0%)]	Train Loss: 0.000411
Train Epoch: 24 [10/43 (23%)]	Train Loss: 0.006735
Train Epoch: 24 [20/43 (47%)]	Train Loss: 0.022878
Train Epoch: 24 [30/43 (70%)]	Train Loss: 0.009693
Train Epoch: 24 [40/43 (93%)]	Train Loss: 0.000555

Train set: Average loss: 0.0036, Accuracy: 408/424 (96%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [9.49288608e-13 6.83397651e-02 1.92361046e-02 4.06563304e-05
 2.76809118e-07 7.09947032e-11 2.22154204e-02 9.50847209e-07
 5.34090816e-08 6.80257435e-05 3.60691249e-02 1.71231989e-08
 2.72108155e-04 5.30340731e-06 2.50909688e-05 6.54292701e-13
 2.58895398e-07 4.51175272e-02 1.38662401e-07 4.91218828e-03
 7.96223844e-07 

Train Epoch: 27 [0/43 (0%)]	Train Loss: 0.000509
Train Epoch: 27 [10/43 (23%)]	Train Loss: 0.105883
Train Epoch: 27 [20/43 (47%)]	Train Loss: 0.000840
Train Epoch: 27 [30/43 (70%)]	Train Loss: 0.000664
Train Epoch: 27 [40/43 (93%)]	Train Loss: 0.000454

Train set: Average loss: 0.0001, Accuracy: 414/424 (98%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [5.18373122e-11 7.93274585e-03 3.44226937e-05 2.03678486e-04
 8.25742035e-08 5.53057378e-10 9.60201491e-04 7.29809790e-06
 9.39796973e-06 4.20426310e-04 2.96188649e-02 2.27838974e-08
 8.68113711e-04 5.49972523e-04 4.62058051e-06 3.43813832e-11
 8.11113841e-06 5.77609278e-02 8.79088375e-08 1.93045940e-03
 2.55099508e-06 

Train Epoch: 30 [0/43 (0%)]	Train Loss: 0.000698
Train Epoch: 30 [10/43 (23%)]	Train Loss: 0.000206
Train Epoch: 30 [20/43 (47%)]	Train Loss: 0.000354
Train Epoch: 30 [30/43 (70%)]	Train Loss: 0.000178
Train Epoch: 30 [40/43 (93%)]	Train Loss: 0.011901

Train set: Average loss: 0.0013, Accuracy: 416/424 (98%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [3.85781025e-11 1.33679353e-03 8.88498253e-05 7.18651281e-05
 1.67796479e-08 2.05400988e-10 4.18629817e-04 3.33034109e-06
 1.02848833e-06 2.46915763e-04 4.10397314e-02 2.38103279e-08
 5.46348492e-05 3.81215687e-05 1.06656153e-05 1.96912920e-11
 1.50424985e-05 3.96139622e-01 2.06833768e-08 5.44110022e-04
 1.21895553e-07 